In [207]:
import pandas as pd
import json
import re

# 假设你的JSON文件名为 'data.json'
file_path = './output.json'

# 使用 pandas 的 read_json 方法读取 JSON 文件
df = pd.read_json(file_path)

# 计算每一列中非空元素的比例
non_empty_proportions = df.notna().mean()

# 找出比例小于30%的列
columns_with_more_than_90_percent_non_empty = non_empty_proportions[non_empty_proportions < 0.5]

# 获取这些列的名字
column_names = columns_with_more_than_90_percent_non_empty.index.tolist()
exceptList = ["无线充电", ""]
column_names = [x for x in column_names if x not in exceptList]
print(column_names)
print(df['无线充电'].value_counts().sum())
print(df['机身材质分类'].value_counts().sum())


# 删除这些列
df.drop(columns=column_names,inplace=True)

# 计算每一列中非空元素的数量
non_empty_counts = df.notna().sum()

# 按照非空元素数量降序排列
sorted_non_empty_counts = non_empty_counts.sort_values(ascending=False)


p = re.compile(r'(;;;(，|游戏运行.{2}|手机性能排行|更多.*?手机＞{0,1}|查看.{2,3}))|>')

# 1. 将所有列表元素用 join 方法 ";;;" 拼接成字符串
# 2. 用正则清除掉多余字符
df = df.map(lambda x: p.sub("",';;;'.join(x)) if isinstance(x,list) else x)


['防抖功能', '变焦倍数', '红外功能', '后置拍照功能', '触摸屏类型', '前置拍照功能', '其他网络参数', '扬声器', '其他屏幕参数', '前置视频拍摄', '其他外观参数', '对焦方式', '无线反向充电', '传感器型号', '三防功能', '连接与共享', '后置视频拍摄', '触控采样率', '镜头片数', '屏幕亮度', 'HDR技术', '焦距', '联名品牌', '散热', '广角', '其他摄像头参数', '对比度', '屏幕盖板', '传感器类型', '5G网络', '扩展容量', '其他功能', '振动马达', '其他硬件参数', '传感器尺寸', '游戏功能', '微距', '3G网络', '屏幕厂商', '指纹识别工艺', '音频支持', '4G网络', '视频支持', '屏幕技术', '续航时间', '摄像头特色', '图片支持', '服务特色', '制作工艺', '多媒体技术', '可选配件', '国外发布时间', '主 屏幕色彩', '产品尺寸 长度', '主 像素密度', '主 屏幕材质', '前置摄像头1 前置摄像头', '前置摄像头1 光圈', '主 屏占比', '后置摄像头1 后置摄像头', '主 屏幕类型', '产品尺寸 厚度', '产品尺寸 宽度', '主 屏幕尺寸', '主 触摸屏类型', '手机类型', '前置摄像头1 传感器型号', '主 分辨率', '主 屏幕比例', '前置摄像头1 对焦方式', '前置摄像头1 防抖功能', '主 屏幕刷新率', '前置摄像头1 广角', '主 其他屏幕参数', '主 触控采样率', '后置摄像头1 摄像头名称', '后置摄像头1 光圈', '前置摄像头1 摄像头名称', '双11', '电池是否可拆卸', '机身材质分类', '店铺', '耳机接口类型', '商品毛重', '屏幕材质类型', '货号', 'CPU核数', '拍照特点', '后置摄像头', 'CPU品牌', '前置摄像头像素', '运营商标志或内容', 'RAM', '机身宽度（mm）', '商品名称', '主屏幕尺寸（英寸）', '商品编号', '充电接口类型', '摄像头光圈大小', '入网型号', 'RAM类型', '前置摄像头', '前摄光圈大小', '可用空间', '电池容量（m

In [173]:
# 3. NFC nan 元素赋值为 None
df.loc[df['NFC'].isna(), 'NFC'] = None
p = re.compile(r'Sumsung Pay|卡模拟|点对点|读卡器|多功能NFC|全方位感应|全功能NFC|全场景')
def clean(x):
    if x is None or "不支持NFC" in x:
        return "无"
    else:
        result = "支持NFC"
        if p.search(x):
            result =  result+"({})".format(",".join(p.findall(x)))
        return result
        
df['NFC'] = df['NFC'].map(clean)


NFC
支持NFC                 1637
无                     1517
支持NFC(读卡器,点对点,卡模拟)     136
支持NFC(全功能NFC)           26
支持NFC(多功能NFC)           21
支持NFC(读卡器,卡模拟)          20
支持NFC(读卡器)              19
支持NFC(全场景)              19
支持NFC(点对点,读卡器,卡模拟)       8
支持NFC(Sumsung Pay)       7
支持NFC(全方位感应)             4
Name: count, dtype: int64


In [208]:
# 5. 无线非 nan 元素赋值为 "多少功率"
p = re.compile(r'(\d{1,2}\.{0,2}\d{0,2})')
def clean(x):
    if p.search(x):
        return "{}W".format(p.search(x).group()) 
    else:
        return "支持NFC"
df['无线充电'] = df.loc[df['无线充电'].notna(), '无线充电'].map(clean)

# {k: v for k, v in data.items() if any(item in k for item in posNameSet)}
# allPosNames

In [ ]:
# 5. 定位导航非 nan 元素赋值为 "哪几种导航技术"
p = re.compile(r'A{0,1}GPS|A{0,1}GLONASS|AGNSS|GNSS|LTE[EP]P|SUPL|FLONASS|SENSOR|格[罗洛]纳斯|QZSS|Beidou|北斗|ABDS|BDS|无线局域网|无线|数据|蜂窝|WLAN|wifi|伽利略|G{0,1}alileo{0,1}|SBAS|iBeacon|电子罗盘|电子指南针|数字指南针|Navic|Navlc|IRNSS|INAVC|AGNSS|IRNSSA|地理围栏',flags=re.I)
p2 = re.compile(r';;;|/|，|、|｜|\|')
# 清除字符
p3 = re.compile(r' |[\?:（）\(\)：\r\n-、/，\+丨;；=]|网络|导航|支持|技术|内置|卫星|双(向|天线|GPS)|其他|包括|，（L1单频|单频{1,2}|：|基站|IP68IP78双3D识别|可|辅助定位|定位|系统|导轨|导航|惯性|微')
p4 = re.compile(r'(^;;;)|(;;;$)',flags=re.I) #开头或结尾是;;;
p5 = re.compile(r';;;;;;',flags=re.I)
p6 = re.compile(r'(A)(;;;)(GPS|GLONASS)',flags=re.I)

posNameSet = ("GPS", "GNSS", "QZSS", "FLONASS", "IRNSS", "SBAS", "IBEACON", "LTEPP", "北斗", "伽利略", "格洛纳斯", "网络定位", "指南针")
patternDict = {
    "辅助全球导航卫星系统":re.compile(r'AGNSS',flags=re.I),
    "全球导航卫星系统":re.compile(r'GNSS',flags=re.I),
    "辅助全球卫星定位系统":re.compile(r'AGPS',flags=re.I),
    "全球卫星定位系统":re.compile(r'GPS',flags=re.I),
    "北斗":re.compile(r'A{0,1}BDS|BEIDOU',flags=re.I),
    "伽利略":re.compile(r'G{0,1}alileo{0,1}',flags=re.I),
    "准天顶卫星系统":re.compile(r'QZSS',flags=re.I),
    "格洛纳斯":re.compile(r'格[罗洛]纳斯|A{0,1}[FG]LONASS',flags=re.I),
    "网络定位":re.compile(r'无线局域网|无线|数据|蜂窝|WLAN|wifi|LBS|地理围栏',flags=re.I),
    "指南针":re.compile(r'(数字|电子)指南针|电子罗盘',flags=re.I),
    "印度定位":re.compile(r'(NavICL(?!\d)|NAV[IL]C|IRNSS|INAVC)(NA){0,1}',flags=re.I),
    "基站定位":re.compile(r'LTE[EP]P',flags=re.I),
    "安全用户平面定位":re.compile(r'SUPL',flags=re.I),
    "蓝牙技术定位":re.compile(r'IBEACON',flags=re.I),
    "卫星增强系统":re.compile(r'SBAS',flags=re.I),
    "传感器定位":re.compile(r'SENSOR',flags=re.I),
    
}
p9 = re.compile(r'[双三四]频',flags=re.I)
p3 = re.compile(r'[ \?:（）\(\)：\r\n、/，\+\-丨;；=]|Ip68 ip78  双3D识别|[双三四]频O{0,1}|卫星（L1\+L5），精度3米|LTEEPSUPL|双(向|天线|GPS)|支持|技术|[微导航轨可]|网络|内置|卫星|其他|包括|系统|精度3米|基站|定位|辅助|惯性|单频+',flags=re.I)
pBand = re.compile(r'L1|L1C|L2|L3|L5|G1|E1|E2|E5[AB]{0,1}|E6|B1|B2|B3',flags=re.I)
pAZ09 = re.compile(r'[a-z0-9]',flags=re.I)
bandsNum = {
    2:"双频",
    3:"三频",
    4:"四频",
}
allPosNames = set()
data = {}
def filter_null(x):
    return x not in ["", "L1L5双频","IP68IP78双3D识别"]
def clean(x):
    ox = x
    if isinstance(x,str):
        if "压力感应，重力加速度，陀螺仪/指纹传感器，距离传感器，光敏感应，电子罗盘/红外遥控/线性马达，NFC" in x:
            return None
        x = p3.sub("",x.upper())
        for i in p.findall(x):
            x = x.replace(i,";;;"+i)
        x2 = "{}".format(";;;".join(p2.split(x)))
        x2 = p5.sub(";;;",x2)
        for i in p6.findall(x2):
            x2 = x2.replace(''.join(i),";;;{}".format(i[0]+i[2]))
        # 统一卫星系统名称
        for k,v in patternDict.items():
            x2 = v.sub(k, x2)
        # 将双频之类的限定词重新统计再后置
        d = list(filter(filter_null,p2.split(x2)))
        for i in range(len(d)):
            band_list = pBand.findall(d[i])
            bandNum = len(band_list)
            s = pAZ09.sub("", d[i])
            allPosNames.add(s)
            if bandNum>1:
                d[i] = d[i] + bandsNum[bandNum]
        x2 = ";;;".join(d)
        x2 = p4.sub("", x2)
        if x2 in ["", "L1L5双频","IP68IP78双3D识别"]:
            return None
        for item in x2.split(';;;'):
            if item=="格洛纳斯A":
                print(x2)
            if item not in data:
                data[item] = 0 
            data[item] = data[item]+1
        return x2
df['定位导航'] = df.loc[df['定位导航'].notna(), '定位导航'].map(clean)

df.loc[df['定位导航'].isna(), '定位导航']

In [ ]:
df.to_json("output_clean.json", orient="records", force_ascii=False)
